In [1]:
!pip install paddlepaddle paddleocr openai pandas openpyxl
!pip install pandas openai python-dotenv


In [2]:
from paddleocr import PaddleOCR
import pandas as pd
import openai
import os
import re
from datetime import datetime


In [3]:
ocr = PaddleOCR(lang='pt', use_angle_cls=True)

def extrair_texto_com_coordenadas(caminho_imagem):
    """Extrai texto e coordenadas da imagem usando PaddleOCR."""
    resultado = ocr.ocr(caminho_imagem, cls=True)
    dados = []
    for linha in resultado[0]:
        coordenadas = linha[0]
        texto = linha[1][0]
        dados.append({'coordenadas': coordenadas, 'texto': texto})
    return dados


[2024/10/27 01:02:57] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/latin/latin_PP-OCRv3_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25,

Extração CPF

In [4]:
def identificar_cpf(dados_extraidos):
    """Identifica o CPF nos textos extraídos."""
    for item in dados_extraidos:
        texto = item['texto']
        if re.match(r'\d{3}\.\d{3}\.\d{3}-\d{2}', texto):
            return texto.strip()
    return "CPF não encontrado"


Extração Nome

In [5]:
def verificar_coordenadas(coordenadas, x_min, x_max, y_min, y_max):
    """Verifica se as coordenadas estão dentro dos limites definidos."""
    for ponto in coordenadas:
        x, y = ponto
        if not (x_min <= x <= x_max and y_min <= y <= y_max):
            return False
    return True

def buscar_nome_com_loop(dados_extraidos):
    """Busca um nome válido dentro de uma faixa de coordenadas."""
    for x_min in range(100, 140):
        for x_max in range(x_min + 1, 400):
            for y_min in range(110, 140):
                for y_max in range(y_min + 1, 170):
                    for item in dados_extraidos:
                        coordenadas = item['coordenadas']
                        texto = item['texto']
                        if verificar_coordenadas(coordenadas, x_min, x_max, y_min, y_max) and len(texto) > 5:
                            return texto
    return "Nome não encontrado"


Extração data de nascimento

In [6]:
def validar_data_nascimento(data_nascimento):
    """Valida se a diferença de anos entre a data e o ano atual é maior que 10."""
    try:
        data = datetime.strptime(data_nascimento, "%d/%m/%Y")
        ano_atual = datetime.now().year
        return (ano_atual - data.year) > 10
    except ValueError:
        return False

def buscar_data_nascimento_com_loop(dados_extraidos):
    """Busca uma data de nascimento válida nas coordenadas especificadas."""
    for x_min in range(490, 650):
        for x_max in range(x_min + 1, 760):
            for y_min in range(200, 270):
                for y_max in range(y_min + 1, 300):
                    for item in dados_extraidos:
                        coordenadas = item['coordenadas']
                        texto = item['texto']
                        if verificar_coordenadas(coordenadas, x_min, x_max, y_min, y_max):
                            if re.match(r'\d{2}/\d{2}/\d{4}', texto) and validar_data_nascimento(texto):
                                return texto
    return "Data de nascimento não encontrada"


Processar Todas as CNHs e Extrair Informações

In [7]:
caminho_pasta = "/content/cnh"
todas_as_informacoes = []

for arquivo in os.listdir(caminho_pasta):
    if arquivo.endswith(('.jpg', '.png', '.jpeg')):
        caminho_imagem = os.path.join(caminho_pasta, arquivo)
        dados_extraidos = extrair_texto_com_coordenadas(caminho_imagem)

        nome = buscar_nome_com_loop(dados_extraidos)
        cpf = identificar_cpf(dados_extraidos)
        data_nascimento = buscar_data_nascimento_com_loop(dados_extraidos)

        todas_as_informacoes.append({
            'arquivo': arquivo,
            'nome': nome,
            'cpf': cpf,
            'data_nascimento': data_nascimento
        })

df_cnh = pd.DataFrame(todas_as_informacoes)
df_cnh['data_extracao'] = datetime.today().strftime('%Y-%m-%d')
print(df_cnh)


[2024/10/27 01:03:07] ppocr DEBUG: dt_boxes num : 38, elapsed : 0.3764381408691406
[2024/10/27 01:03:07] ppocr DEBUG: cls num  : 38, elapsed : 0.11763525009155273
[2024/10/27 01:03:09] ppocr DEBUG: rec_res num  : 38, elapsed : 1.6550533771514893
[2024/10/27 01:03:21] ppocr DEBUG: dt_boxes num : 48, elapsed : 0.213090181350708
[2024/10/27 01:03:21] ppocr DEBUG: cls num  : 48, elapsed : 0.1021888256072998
[2024/10/27 01:03:25] ppocr DEBUG: rec_res num  : 48, elapsed : 3.387603998184204
[2024/10/27 01:03:37] ppocr DEBUG: dt_boxes num : 46, elapsed : 0.19790101051330566
[2024/10/27 01:03:37] ppocr DEBUG: cls num  : 46, elapsed : 0.11648416519165039
[2024/10/27 01:03:39] ppocr DEBUG: rec_res num  : 46, elapsed : 1.8049798011779785
[2024/10/27 01:03:52] ppocr DEBUG: dt_boxes num : 45, elapsed : 0.3746354579925537
[2024/10/27 01:03:52] ppocr DEBUG: cls num  : 45, elapsed : 0.11721968650817871
[2024/10/27 01:03:54] ppocr DEBUG: rec_res num  : 45, elapsed : 1.7167260646820068
           arquivo

Salvar em Excel Incremental

In [8]:
def salvar_excel_incremental(df_novo, caminho_excel='dados_incrementais.xlsx'):
    try:
        df_existente = pd.read_excel(caminho_excel)
        df_atualizado = pd.concat([df_existente, df_novo], ignore_index=True)
    except FileNotFoundError:
        df_atualizado = df_novo

    df_atualizado.to_excel(caminho_excel, index=False)

salvar_excel_incremental(df_cnh)


Verificar e Atualizar CPFs no Excel

In [11]:
def atualizar_doc1(df_cnh, caminho_doc1='/content/arquivo/doc1.xlsx'):
    """Atualiza e acrescenta informações de CNHs no doc1.xlsx."""
    try:
        # Ler o Excel existente
        df_doc1 = pd.read_excel(caminho_doc1)
    except FileNotFoundError:
        # Se o Excel não existir, cria um DataFrame vazio
        df_doc1 = pd.DataFrame(columns=['cpf', 'nome', 'local', 'data_nascimento', 'data_extracao'])

    # Lista para armazenar novas CNHs que não estão no doc1.xlsx
    novas_cnhs = []

    # Iterar sobre as CNHs extraídas
    for _, linha in df_cnh.iterrows():
        cpf = linha['cpf']

        # Se o CPF já está no doc1, atualiza as informações de local e data
        if cpf in df_doc1['cpf'].values:
            df_doc1.loc[df_doc1['cpf'] == cpf, ['data_nascimento', 'data_extracao']] = linha['data_nascimento'], linha['data_extracao']
        else:
            # Se o CPF não estiver no doc1, adiciona a CNH à lista de novas CNHs
            novas_cnhs.append(linha)

    # Adicionar novas CNHs ao DataFrame existente
    if novas_cnhs:
        df_novas_cnhs = pd.DataFrame(novas_cnhs)
        df_doc1 = pd.concat([df_doc1, df_novas_cnhs], ignore_index=True)

    # Salvar o DataFrame atualizado no Excel
    df_doc1.to_excel(caminho_doc1, index=False)

    print(f"Atualização concluída! Total de registros no doc1.xlsx: {len(df_doc1)}")

# Atualizar ou adicionar novas CNHs no doc1.xlsx
atualizar_doc1(df_cnh)


Atualização concluída! Total de registros no doc1.xlsx: 4


Criar Chatbot com API do ChatGPT

In [33]:
import pandas as pd
import openai
import os
from dotenv import load_dotenv

# Carregar a chave da API do OpenAI
load_dotenv()
openai.api_key = "API_KEY"

# Carregar o Excel em um DataFrame
df = pd.read_excel('/content/arquivo/doc1.xlsx')

def gerar_resposta(pergunta):
    """
    Gera uma resposta usando a API da OpenAI com base nos dados carregados.
    """
    # Converter o DataFrame para string para que o modelo possa usá-lo
    dados_str = df.to_string(index=False)

    # Construir o prompt com os dados e a pergunta do usuário
    prompt = f"""
    Abaixo estão alguns dados em formato tabular:

    {dados_str}

    Baseado nesses dados, responda a seguinte pergunta:
    {pergunta}
    """

    # Chamada à API do OpenAI
    try:
        response = openai.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "Você é um assistente que responde perguntas com base em dados fornecidos."},
                {"role": "user", "content": prompt}
            ]
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Ocorreu um erro: {str(e)}"

# Loop de interação do chatbot
print("Chatbot: Olá! Como posso te ajudar com os dados do Excel?")
while True:
    pergunta = input("Você: ")
    if pergunta.lower() in ['sair', 'exit', 'quit']:
        print("Chatbot: Até mais!")
        break
    resposta = gerar_resposta(pergunta)
    print(f"Chatbot: {resposta}")


Chatbot: Olá! Como posso te ajudar com os dados do Excel?
Você: Qual é a data_nascimento do CPF 916.127.920-06?
Chatbot: A data de nascimento do CPF 916.127.920-06 é 02/02/1956.
Você: sair
Chatbot: Até mais!
